# Introduction 
* For this assignment, you need to benchmark the following models using RMSE, MAE and fit-time / prediction-time:

* * User-based CF with cosine similarity.
* * User-based CF with pearson correlation similarity.
* * Item-based CF with cosine similarity.
* * Item-based CF with pearson correlation similarity.
* * SVD
* * NMF
* Note :
** You should visualize your benchmarking results with a bar chart and to interepret them
** You shouldn't implement the models from scratch, you need to use their surprise implementation.

* Notebook :

* Your notebook should be leasable, well organized and commented. It should contain 3 separate parts :

* [Data loading]
* [Utils]
* [Model benchmarking]

In [15]:
import os
import re
from surprise import Dataset
from pathlib import Path
from surprise import Reader
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD
from surprise import accuracy
from surprise.prediction_algorithms import knns
from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.similarities import cosine, msd, pearson
import time
import pandas as pd 
from tabulate import tabulate

# Data loading

* This is data loading section: The function (load_data) will download the ml-100k file which contains dataset otherwise if we have data we can import by path file

In [16]:
def load_data(load_from_surprise: bool) -> pd.DataFrame:
    if load_from_surprise == True:
        try:
            
            dataframe = Dataset.load_builtin('ml-100k')
            
        except EOFError as e:
            print(e)   
    elif load_from_surprise == False:
        user_input = input("Enter the path of the dataset: ")
        assert os.path.exists(user_input), "I did not find the file at, "+str(user_input)
        filepath = user_input
        reader = Reader(line_format='user item rating timestamp', sep=",", skip_lines= 1)
        dataframe = Dataset.load_from_file(filepath, reader)

    return dataframe

In [17]:
dataframe = load_data(False)

# Utils

In [18]:
def util(algorithm: str,model_kwargs: dict) -> float:
    #Split data
    trainset, testset = train_test_split(dataframe, 0.2, random_state=42)
    # print(len(testset))

    #train model
    if algorithm == 'KNN':
        model = knns.KNNBaseline(sim_options = model_kwargs['sim_options'])
    # elif algorithm == 'grid_KNN':
    #     param_grid = {'K' : model_kwargs['K'], 'sim_options' : model_kwargs['sim_options']}
    #     grid_model = GridSearchCV(knns.KNNBaseline, param_grid, measures=['rmse', 'mae'], cv=3)
    elif algorithm == 'SVD':
        model = SVD(n_factors=model_kwargs['n_factors'], n_epochs=model_kwargs['n_epochs'], lr_all=model_kwargs['lr_all'], reg_all= model_kwargs['reg_all'])
    elif algorithm == 'NMF':
        model = NMF(n_factors=model_kwargs['n_factors'], n_epochs=model_kwargs['n_epochs'])
    else:
        print('Please upload the model with parameters....!')
    

    # Model fit and running time
    start_time = time.time()
    # if algorithm == 'Grid_SVD':
    #     grid_model.fit(dataframe)
    # else:
    model.fit(trainset)
    fit_time = ('model running Fit time is %s seconds' % (round(time.time() - start_time,4)))
    #Model Prediction and running time
    start_time = time.time()
    prediction = model.test(testset)
    pred_time = ('model running Prediction time is %s seconds' % (round(time.time() - start_time,4)))

    # Model Score and running time
    start_time = time.time()
    # if algorithm == 'Grid_SVD':
    #     RMSE_result = grid_model.best_score['rmse']
    #     MAE_result = grid_model.best_score['mae']
    # else:
    RMSE = accuracy.rmse(prediction)
    MAE = accuracy.mse(prediction)
    RMSE_result = ('RMSE %s ' % RMSE)
    MAE_result = ('MAE: %s ' % MAE)
    pred_time = ('model running Score time is %s seconds' % (round(time.time() - start_time,4)))

    return model,fit_time, pred_time, RMSE_result, MAE_result

    
        

# Bench Mark

In [19]:
def benchmark(model_parameters: dict) -> (pd.DataFrame, dict):    
    result_list = []
    for i in model_parameters.keys():        
        items = model_parameters[i]
        model,fit_time, pred_time, RMSE_result, MAE_result = util(algorithm=items['algorithm'],model_kwargs=items['model_kwargs'])
        column = [i, re.findall("\d+\.\d+",fit_time)[0], re.findall("\d+\.\d+",pred_time)[0],re.findall("\d+\.\d+",RMSE_result)[0],re.findall("\d+\.\d+",MAE_result)[0]]
        tabulate([column], tablefmt="pipe")
        result_list.append(column)
    header = ['Name','Model_fit_time','Model prediction time','RMSE','MAE']
    dataframe = pd.DataFrame(result_list)
    dataframe.columns = header        
    return dataframe

In [20]:
plot_benchmark = benchmark(model_parameters= {'KNN (Cosine User based)':
                                               {'algorithm': 'KNN', 'model_kwargs': {'sim_options': {'user_based': True, 'name': 'cosine'}}},
                                               'KNN (Cosine Item based)':
                                               {'algorithm': 'KNN', 'model_kwargs': {'sim_options': {'user_based': False, 'name': 'cosine'}}},
                                               'KNN (Pearson User based)':
                                               {'algorithm': 'KNN', 'model_kwargs': {'sim_options': {'user_based': True, 'name': 'pearson'}}},
                                               'KNN (Pearson Item based)':
                                               {'algorithm': 'KNN', 'model_kwargs': {'sim_options': {'user_based': False, 'name': 'pearson'}}},
                                               'SVD':
                                               {'algorithm': 'SVD', 'model_kwargs':{'n_factors':100, 'n_epochs':10, 'lr_all':0.005,'reg_all':0.4}},
                                               'NMF':
                                               {'algorithm': 'NMF', 'model_kwargs':{'n_factors':15, 'n_epochs': 50}}
                                               })

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8951
MSE: 0.8013
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9030
MSE: 0.8154
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9010
MSE: 0.8117
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8969
MSE: 0.8044
RMSE: 0.9057
MSE: 0.8202
RMSE: 0.9467
MSE: 0.8962


In [21]:
plot_benchmark

,Name,Model_fit_time,Model prediction time,RMSE,MAE
0,KNN (Cosine User based),0.833,0.029,0.8951420019359863,0.8012792036299652
1,KNN (Cosine Item based),8.7619,0.031,0.9029882502122452,0.8153877800213724
2,KNN (Pearson User based),1.095,0.036,0.9009500761521475,0.8117110397185603
3,KNN (Pearson Item based),11.3549,0.043,0.8968999689302629,0.8044295542671066
4,SVD,2.09,0.043,0.9056589927428532,0.8202182111359995
5,NMF,5.3665,0.032,0.9466592377836404,0.8961637124811029


# Grid Search for KNN

In [43]:
param_grid = {'K': [40,60,70], 'sim_options': {'user_based': [True, False], 'name': ['cosine', 'pearson']}}
gs_svd = GridSearchCV(knns.KNNBaseline, param_grid, measures=['rmse', 'mae'], cv=3)
start_time = time.time()
gs_svd.fit(dataframe)
fit_time = (round(time.time() - start_time,4))
# svd = gs_svd.best_estimator['rmse']
RMSE_grid = gs_svd.best_score['rmse']
MSE_grid = gs_svd.best_score['mae']
best_params = gs_svd.best_params

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Com

In [44]:
print(RMSE_grid)
print(MSE_grid)
print(best_params)

0.9018140627833082
0.6906134770766675
{'rmse': {'K': 40, 'sim_options': {'user_based': False, 'name': 'cosine'}}, 'mae': {'K': 40, 'sim_options': {'user_based': False, 'name': 'pearson'}}}


In [45]:
df2 = {"Name":["Grid KNN"],"Model_fit_time":[fit_time],"RMSE": [RMSE_grid],"MAE": [MSE_grid]}

data_frame = pd.DataFrame.from_dict(df2)

# Final result table

In [46]:
plot_benchmark.append(data_frame)

,Name,Model_fit_time,Model prediction time,RMSE,MAE
0,KNN (Cosine User based),0.833,0.029,0.8951420019359863,0.8012792036299652
1,KNN (Cosine Item based),8.7619,0.031,0.9029882502122452,0.8153877800213724
2,KNN (Pearson User based),1.095,0.036,0.9009500761521475,0.8117110397185603
3,KNN (Pearson Item based),11.3549,0.043,0.8968999689302629,0.8044295542671066
4,SVD,2.09,0.043,0.9056589927428532,0.8202182111359995
5,NMF,5.3665,0.032,0.9466592377836404,0.8961637124811029
0,Grid KNN,325.254,NaN,0.901814,0.690613


* The fit time is more than any other model it takes more than 5-10 minutes 

# Ploting the graphs

* For ploting I'm not using the grid search KNN because the fiting time more compare to others that makes graph incredible

In [49]:
import plotly.express as px
fig = px.bar(plot_benchmark, x='Name', y='Model_fit_time', title="Model fit time compare")
fig.show()

In [50]:
import plotly.express as px
fig = px.bar(plot_benchmark, x='Name', y='Model prediction time', title="Model prediction time compare")
fig.show()

In [51]:
fig = px.bar(plot_benchmark, x='Name', y='RMSE', title="Comparing the RMSE score")
fig.show()

In [52]:
fig = px.bar(plot_benchmark, x='Name', y='MAE', title="Comparing the MAE score")
fig.show()